In [ ]:
! pip install hubmap-sdk

In [3]:
from hubmap_sdk import SearchSdk
import json
from collections import defaultdict


# #In this example, the token and service url are being retrieved from a configuration file.
# url = app.config['DEV_URL']
# token = app.config['GLOBUS_TOKEN'] 

service_url = "https://search.api.hubmapconsortium.org/v3/"

search_instance = SearchSdk(service_url=service_url)
# search_instance = SearchSdk(token, url)
searchsdk_instance = search_instance

In [4]:
list_of_indices = searchsdk_instance.indices()
list_of_indices

['entities', 'portal', 'hm_antibodies', 'files']

In [5]:


# searchbody = {
#   "size": 0,
#   "aggs": {
#     "field_values": {
#       "terms": {
#         "field": "entity_type",
#         "size": 10000
#       }
#     }
#   }
# }
searchbody = {
  "aggs": {
    "created_by_user_displayname": {
      "filter": {
        "term": {
          "death_event.keyword": "Motor"
        }
      }
    }
  },
  "_source": []
}
search_result = searchsdk_instance.search_by_index(searchbody, "portal")
search_result

import json

with open("search_result3.json", "w") as f:
    f.write(json.dumps(search_result, indent=2))

In [6]:


searchbody = {
#   "query": {
#     "simple_query_string": {
#       "query": "blunt",
#       "fields": ["donor.*"],
#       "lenient": True,
#     }
#   },
#   "query": {
#     "match_all": {}
#   },
    "query": {
        # "match_all": {},

        # "bool": {
        #     "filter": [
        #         {"term": {"entity_type": "Donor"}},
        #     ],
        # },
        "function_score": {
            "query": {
                "match": {
                    "entity_type": "Dataset"
                }
            },
            "random_score": {
                "seed": 10
            }
        },

    },
# "aggs": {
#     "entity_type": {
#       "terms": { "field": "entity_type" }
#     }
#   },
#   "fields": [
# #   "_source": [
#     "hubmap_id",
#     "group_name",
#     "mapped_data_types",
#     "origin_samples.mapped_organ",
#     "mapped_status",
#     "mapped_last_modified_timestamp",
#     "last_modified_timestamp",
#     "descendant_counts.entity_type",
#     "thumbnail_file.file_uuid",
#     "uuid",
#     "donor"
#   ],
  "_source": {
    "excludes": [ 
        "ancestors",
        "descendants",
        "immediate_ancestors",
        "immediate_descendants",
        "metadata",
        "donor.metadata"
    ],
    # "includes": [
    #     "hubmap_id",
    #     "entity_type",
    # ]
  },
  "size": 100,
}
search_result = searchsdk_instance.search_by_index(searchbody, "portal")
with open("search_result6.json", "w") as f:
    f.write(json.dumps(search_result, indent=2))

In [28]:
field_example_dict = defaultdict(lambda: defaultdict(int))

# iterate recursively through the json structure
# keys should be like "hits.hits.n._source.donor.mapped_metadata.age_value"

def get_field_example_dict(json_obj, field_example_dict, prefix=""):
    if isinstance(json_obj, dict):
        for key in json_obj:
            if prefix == "":
                new_prefix = key
            else:
                new_prefix = prefix + "." + key
            get_field_example_dict(json_obj[key], field_example_dict, new_prefix)
    elif isinstance(json_obj, list):
        for i, item in enumerate(json_obj):
            new_prefix = prefix + "[n]"
            get_field_example_dict(item, field_example_dict, new_prefix)
    else:
        field_example_dict[prefix][json_obj] += 1


# get_field_example_dict(search_result, field_example_dict)
for hit in search_result["hits"]["hits"]:
    get_field_example_dict(hit["_source"], field_example_dict)

field_example_dict_of_list = {k: v for k, v in field_example_dict.items()}

In [29]:
with open("es_field_samples_dataset.json", "w") as f:
    f.write(json.dumps(field_example_dict_of_list, indent=2))

In [17]:


searchbody = {
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "provider_info": "Stanford"
          }
        },
        {
          "match": {
            "entity_type": "Dataset"
          }
        }
      ]
    }
  },
  "_source": {
    "excludes": [ 
        "ancestors",
        "descendants",
        "immediate_ancestors",
        "immediate_descendants",
        "metadata",
        "donor.metadata"
    ],
    "includes": [
        "donor.mapped_metadata.death_event",
        "donor.mapped_metadata.mechanism_of_injury",
        "donor.mapped_metadata.sex",
        "donor.mapped_metadata.age_value",
        "donor.mapped_metadata.race",
        "provider_info",
        # "files.type",
        # "source_samples.created_by_user_email",
        "donor.mapped_metadata.medical_history"
    ]
  },
    "size": 100,
}
search_result = searchsdk_instance.search_by_index(searchbody, "portal")
with open("search_result8.json", "w") as f:
    f.write(json.dumps(search_result, indent=2))

In [13]:
print(f"{service_url}portal/_mapping")

https://search.api.hubmapconsortium.org/v3/portal/_mapping


In [14]:
!curl -X GET https://search.api.hubmapconsortium.org/v3/portal/_mapping

{"message": "Unable to find the requested resource"}

In [16]:
status_object = search_instance.status()
status_object

build: main:4c9125a, elasticsearch_connection: True, elasticsearch_status: green, version: 3.2.0


{'build': 'main:4c9125a',
 'elasticsearch_connection': True,
 'elasticsearch_status': 'green',
 'version': '3.2.0'}

In [23]:
search_result.keys()

dict_keys(['_shards', 'hits', 'timed_out', 'took'])

In [30]:
import webbrowser

url = "https://portal.hubmapconsortium.org/browse/dataset/2107df00633f703d39e1ec74c271a9e5"

webbrowser.open(url, new=0, autoraise=True)

True

In [108]:
import json

# Load the Elasticsearch response from result.json
with open("search_result5.json", "r") as file:
    response = json.load(file)

# Extract relevant information from the response
hits = response["hits"]["hits"]
total_hits = response["hits"]["total"]["value"]
unique_fields = set()

# Analyze each hit to gather unique field names
for hit in hits:
    fields = hit["_source"].keys()
    unique_fields.update(fields)

# Print the total number of hits and unique field names
print(f"Total Hits: {total_hits}")
print(f"Unique Fields: {unique_fields}")

Total Hits: 3902
Unique Fields: {'anatomy_1', 'protocol_url', 'mapped_organ', 'next_revision_uuid', 'mapped_last_modified_timestamp', 'data_access_level', 'created_by_user_email', 'mapped_status', 'donor', 'status', 'sample_category', 'anatomy_2', 'contributors', 'description', 'group_uuid', 'files', 'hubmap_id', 'entity_type', 'index_version', 'lab_tissue_sample_id', 'organ', 'previous_revision_uuid', 'rui_location', 'label', 'created_timestamp', 'doi_url', 'lab_dataset_id', 'lab_donor_id', 'descendant_counts', 'mapper_metadata', 'mapped_sample_category', 'display_subtype', 'immediate_descendants', 'last_modified_timestamp', 'registered_doi', 'data_types', 'immediate_ancestors', 'origin_samples_unique_mapped_organs', 'source_samples', 'group_name', 'thumbnail_file', 'datasets', 'mapped_metadata', 'creators', 'ancestor_counts', 'mapped_data_types', 'created_by_user_displayname', 'uuid', 'portal_metadata_upload_files', 'descendant_ids', 'contains_human_genetic_sequences', 'mapped_data_a

In [110]:
import random

# apple, banana, pear, cup, keyboard, mouse
objects = [
    # "apple", 
    "banana", 
    # "pear", 
    "cup", 
    # "keyboard", 
    "headphones"
    ]

# shuffled
random.shuffle(objects)

print(objects)

['banana', 'cup', 'headphones']
